## Importing Libraries

In [ ]:
import shutil
shutil.rmtree("/content/sample_data")

In [ ]:
! pip install mediapipe
! pip install sk-video
! pip install keras-video-generators
import numpy as np 
import pandas as pd 
import os
import skvideo.io
import cv2 
import mediapipe as mp
import time
import matplotlib.pyplot as plt
import glob
import keras
import tensorflow
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM,Flatten, TimeDistributed, Conv2D, Dropout,Input

In [ ]:
from keras_video import VideoFrameGenerator

In [ ]:
from pipeline import pose_estimation,video_array_maker

## Downloading and Unzipping Data

In [ ]:
# use sub directories names as classes
"""
Greetings_1of2 : Hello ,How are you,Alright ,Good morning ,Good Afternoon  :5
Greetings_2of2 : Thank You   :6
Pronounts_1of2: You ,He, It  :9
Pronounts_2of2:They : 10
None : 11
Adjectives_1of8 : Loud ,Happy ,Sad ,Beautiful ,quiet : 16
"""

'\nGreetings_1of2 : Hello ,How are you,Alright ,Good morning ,Good Afternoon  :5\nGreetings_2of2 : Thank You   :6\nPronounts_1of2: You ,He, It  :9\nPronounts_2of2:They : 10\nNone : 11\nAdjectives_1of8 : Loud ,Happy ,Sad ,Beautiful ,quiet : 16\n'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#unzip realtime data

In [ ]:
! wget https://zenodo.org/record/4010759/files/Greetings_1of2.zip
! wget https://zenodo.org/record/4010759/files/Greetings_2of2.zip
! wget https://zenodo.org/record/4010759/files/Pronouns_1of2.zip
! wget https://zenodo.org/record/4010759/files/Pronouns_2of2.zip
! wget https://zenodo.org/record/4010759/files/Adjectives_1of8.zip


In [ ]:
! unzip Greetings_1of2.zip
! unzip Greetings_2of2.zip
!unzip Pronouns_1of2.zip
!unzip Pronouns_2of2.zip
!unzip Adjectives_1of8.zip

Archive:  Greetings_1of2.zip
   creating: Greetings/
   creating: Greetings/48. Hello/
  inflating: Greetings/48. Hello/MVI_0029.MOV  
  inflating: Greetings/48. Hello/MVI_0030.MOV  
  inflating: Greetings/48. Hello/MVI_0031.MOV  
  inflating: Greetings/48. Hello/MVI_0032.MOV  
  inflating: Greetings/48. Hello/MVI_0037.MOV  
  inflating: Greetings/48. Hello/MVI_0038.MOV  
  inflating: Greetings/48. Hello/MVI_0039.MOV  
  inflating: Greetings/48. Hello/MVI_0089.MOV  
  inflating: Greetings/48. Hello/MVI_0090.MOV  
  inflating: Greetings/48. Hello/MVI_0091.MOV  
  inflating: Greetings/48. Hello/MVI_9914.MOV  
  inflating: Greetings/48. Hello/MVI_9915.MOV  
  inflating: Greetings/48. Hello/MVI_9916.MOV  
  inflating: Greetings/48. Hello/MVI_9917.MOV  
  inflating: Greetings/48. Hello/MVI_9954.MOV  
  inflating: Greetings/48. Hello/MVI_9955.MOV  
  inflating: Greetings/48. Hello/MVI_9956.MOV  
  inflating: Greetings/48. Hello/MVI_9957.MOV  
  inflating: Greetings/48. Hello/MVI_9982.MOV  
 

In [ ]:
shutil.rmtree("/content/Pronouns/40. I")
shutil.rmtree("/content/Pronouns/46. you (plural)")
shutil.rmtree("/content/Pronouns/43. she")
shutil.rmtree("/content/Pronouns/45. we")

In [ ]:
shutil.rmtree("/content/Greetings/53. Good evening")
shutil.rmtree("/content/Greetings/54. Good night")
shutil.rmtree("/content/Greetings/56. Pleased")

In [ ]:
shutil.rmtree("/content/Adjectives/6. Ugly")
shutil.rmtree("/content/Adjectives/7. Deaf")
shutil.rmtree("/content/Adjectives/8. Blind")

In [ ]:
os.makedirs("/content/Input")

In [ ]:
# shutil.rmtree("/content/Adjectives")
# shutil.rmtree("/content/Greetings")
# shutil.rmtree("/content/Pronouns")
# shutil.rmtree("/content/Input")

### Adding our Own Data

In [ ]:
wordtype=["Greetings","Pronouns","Adjectives"]

for i in wordtype:
   for dirpath,dirfile,filename in os.walk(os.path.join("/content",i)):
     if len(dirpath.split("/"))==4:
       word ="".join([i for i in dirpath.split("/")[3] if i.isalpha() ])
       word=word.rstrip()
       os.rename(dirpath,"/content/"+dirpath.split("/")[2]+"/"+word[0].upper()+word[1:])

In [ ]:
d=['/content/Greetings',"/content/Pronouns/","/content/Adjectives"]
target_dir = '/content/Input'

for i in d:
    file_names = os.listdir(i)
    for file_name in file_names:
        shutil.move(os.path.join(i, file_name), target_dir)

In [ ]:
i=0
for dirpath,dirfile,filename in os.walk("/content/realtime_data"):
 if(i==0):
   i+=1
   continue
 #print(dirpath)
 #print(filename)
 target_dir = '/content/Input'+"/"+dirpath.split("/")[-1]
 for j in filename:
    #print(dirpath+"/"+j)
    #print(target_dir)
    shutil.copy(dirpath+"/"+j,target_dir)

In [ ]:
os.makedirs("/content/Output")

## Applying Holistic Model on Video and Resizing it.

In [ ]:
for dirpathed,dirnamed,file in os.walk("/content/Input"):
   z=0
   for class_ in dirnamed:
        for dirpath,dirname,files in os.walk(os.path.join(dirpathed,class_)):
              for i in files:
                    pather=os.path.join(dirpath,i)
                    print(pather)

In [ ]:
for dirpathed,dirnamed,file in os.walk("/content/Input"):
    for class_  in dirnamed:
      os.makedirs(os.path.join("/content/Output",class_))

FileExistsError: ignored

In [ ]:
z=0
for dirpathed,dirnamed,file in os.walk("/content/Input"):
 for class_  in dirnamed:
   for dirpath,dirname,files in os.walk(os.path.join(dirpathed,class_)):
     for i in files:
          pather=os.path.join(dirpath,i)
          z+=1
          print("..{} th Video..".format(z))
          video_array_maker(pather)
   print(".............{} over..........".format(class_))

..1 th Video..
..2 th Video..
..3 th Video..
..4 th Video..
..5 th Video..
..6 th Video..
..7 th Video..
..8 th Video..
..9 th Video..
..10 th Video..
..11 th Video..
..12 th Video..
..13 th Video..
..14 th Video..
..15 th Video..
..16 th Video..
..17 th Video..
..18 th Video..
..19 th Video..
..20 th Video..
..21 th Video..
..22 th Video..
..23 th Video..
..24 th Video..
..25 th Video..
..26 th Video..
..27 th Video..
..28 th Video..
..29 th Video..
..30 th Video..
..31 th Video..
..32 th Video..
..33 th Video..
..34 th Video..
..35 th Video..
..36 th Video..
..37 th Video..
..38 th Video..
..39 th Video..
..40 th Video..
..41 th Video..
..42 th Video..
..43 th Video..
..44 th Video..
..45 th Video..
..46 th Video..
.............Loud over..........
..47 th Video..
..48 th Video..
..49 th Video..
..50 th Video..
..51 th Video..
..52 th Video..
..53 th Video..
..54 th Video..
..55 th Video..
..56 th Video..
..57 th Video..
..58 th Video..
..59 th Video..
..60 th Video..
..61 th Video..


## Video Augmentation

In [ ]:
classes = [i.split(os.path.sep)[2] for i in glob.glob('./Output/*')]
classes

['Loud',
 'They',
 'Sad',
 'Quiet',
 'He',
 'Thank you',
 'How are you',
 'You',
 'It',
 'Good Afternoon',
 'Hello',
 'Alright',
 'Beautiful',
 'Happy',
 'None',
 'Good Morning']

In [ ]:

# ImageDataGenerator
datagen = tensorflow.keras.preprocessing.image.ImageDataGenerator(
        rotation_range=10, # rotation
        width_shift_range=0.3, # horizontal shift
        height_shift_range=0.2, # vertical shift
        zoom_range=0.2, # zoom
        horizontal_flip=True, # horizontal flip
        brightness_range=[0.8,1.5]) # brightness

In [ ]:

# some global params
SIZE = (224,224)
CHANNELS = 3
NBFRAME = 45
BS = 4
# pattern to get videos and classes
glob_pattern='./Output/{classname}/*'
# for data augmentation

# Create video frame generator
train = VideoFrameGenerator(
    classes=classes, 
    glob_pattern=glob_pattern,
    nb_frames=NBFRAME,
    split_val=.1, 
    shuffle=True,
    batch_size=BS,
    target_shape=SIZE,
    nb_channel=CHANNELS,
    transformation=datagen,
    use_frame_cache=False)

class Alright, validation count: 8, train count: 36
class Beautiful, validation count: 5, train count: 24
class Good Afternoon, validation count: 9, train count: 38
class Good Morning, validation count: 8, train count: 35
class Happy, validation count: 8, train count: 35
class He, validation count: 9, train count: 38
class Hello, validation count: 8, train count: 34
class How are you, validation count: 8, train count: 34
class It, validation count: 8, train count: 36
class Loud, validation count: 9, train count: 37
class None, validation count: 4, train count: 19
class Quiet, validation count: 4, train count: 17
class Sad, validation count: 6, train count: 24
class Thank you, validation count: 10, train count: 41
class They, validation count: 8, train count: 34
class You, validation count: 8, train count: 36
Total data: 16 classes for 518 files for train


## Training Model

In [ ]:
model=None

In [ ]:
mobilenet = tf.keras.applications.mobilenet.MobileNet(
        include_top=False,
        input_shape=(224,224,3),
        weights='imagenet')
    # Keep 9 layers to train
trainable = 14
for layer in mobilenet.layers[:-trainable]:
    layer.trainable = False
for layer in mobilenet.layers[-trainable:]:
    layer.trainable = True

17235968/17225924 [==============================] - 0s 0us/step


In [ ]:
model=Sequential()
model.add(TimeDistributed(mobilenet ,input_shape=(45,224,224,3)))
model.add(TimeDistributed( Flatten()))
model.add(LSTM(128, activation='relu', return_sequences=False)) 
model.add(Dense(64,activation="relu"))
model.add(Dense(16,activation="softmax")) 
model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(), metrics=tf.keras.metrics.Accuracy())

In [ ]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_8 (TimeDis  (None, 45, 7, 7, 1024)   3228864   
 tributed)                                                       
                                                                 
 time_distributed_9 (TimeDis  (None, 45, 50176)        0         
 tributed)                                                       
                                                                 
 lstm_4 (LSTM)               (None, 128)               25756160  
                                                                 
 dense_8 (Dense)             (None, 64)                8256      
                                                                 
 dense_9 (Dense)             (None, 16)                1040      
                                                                 
Total params: 28,994,320
Trainable params: 27,359,312


In [ ]:
valid = train.get_validation_generator()

Total data: 16 classes for 120 files for validation


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
os.mkdir("/content/checkpoint")

In [ ]:
checkpoint_filepath = '/content/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='train_loss',
    mode='max',
    save_best_only=True)

In [ ]:
#model=None

In [ ]:
model.fit(
    train,
    validation_data=valid,
    verbose=1,
    epochs=25,
    callbacks=model_checkpoint_callback)

Epoch 1/15
129/129 [==============================] - 266s 2s/step - loss: 44455.1680 - accuracy: 0.8749 - val_loss: 102420.3203 - val_accuracy: 0.8859
Epoch 2/15
129/129 [==============================] - 260s 2s/step - loss: 53651.7031 - accuracy: 0.8818 - val_loss: 302301.1250 - val_accuracy: 0.8844
Epoch 3/15
129/129 [==============================] - 259s 2s/step - loss: 44050.9492 - accuracy: 0.8817 - val_loss: 9583.7979 - val_accuracy: 0.8792
Epoch 4/15
129/129 [==============================] - 260s 2s/step - loss: 9905.5625 - accuracy: 0.8777 - val_loss: 2269.9243 - val_accuracy: 0.8719
Epoch 5/15
129/129 [==============================] - 261s 2s/step - loss: 67976.1094 - accuracy: 0.8788 - val_loss: 443661.6562 - val_accuracy: 0.8813
Epoch 6/15
 74/129 [================>.............] - ETA: 1:49 - loss: 125120.7578 - accuracy: 0.8820

KeyboardInterrupt: ignored

## Checking out Model Performance on Unseen Data

In [ ]:
!unzip Tester.zip

In [ ]:
for i,j,k in os.walk("/content/in"):
  for m in k:
    print(os.path.join("/content/in",m))

/content/Tester/WhatsApp Video 2021-05-01 at 8.17.52 PM (1).mp4
/content/Tester/WhatsApp Video 2021-05-01 at 8.17.53 PM.mp4
/content/Tester/WhatsApp Video 2021-05-01 at 8.17.51 PM.mp4
/content/Tester/WhatsApp Video 2021-05-01 at 8.17.52 PM (2).mp4
/content/Tester/WhatsApp Video 2021-05-01 at 8.17.52 PM.mp4
/content/Tester/WhatsApp Video 2021-05-01 at 8.17.51 PM (1).mp4


In [ ]:
for i,j,k in os.walk("/content/in"):
  for m in k:
    pather=os.path.join("in",m)
    video_array_maker(pather,output_directory="out",output_folder="No",remove_input=False)
    print(pather)

In [ ]:
arr=[]
for i,j,k in os.walk("/content/out"):

  for m in k:
    pather=os.path.join("/content/out",m)
    print(pather)
    a=skvideo.io.vread(pather)
    arr.append(a)


arr=np.array(arr)

/content/Tester2/thank you 2.mp4
/content/Tester2/you.mp4
/content/Tester2/nothing.mp4
/content/Tester2/you2.mp4
/content/Tester2/thank you.mp4
/content/Tester2/he.mp4


In [ ]:
arr.shape

(6, 45, 224, 224, 3)

In [ ]:
prediction=model.predict(arr)

In [ ]:
for i in  np.argmax(prediction,axis=1):
  print(classes[i])

47. they
40. I
49. How are you
41. you
49. How are you
50. Alright


In [ ]:
np.argmax(prediction,axis=1)

array([5, 0, 7, 1, 7, 8])

In [ ]:
prediction.shape

(6, 16)

In [ ]:
prediction[5]

array([0.0000000e+00, 1.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 1.0086898e-14],
      dtype=float32)